In [3]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import cvxpy as cp
import scipy.io
%matplotlib inline

In [4]:

f_data = scipy.io.loadmat('flow.mat')["flow"].reshape(28,)
C = scipy.io.loadmat('capacities.mat')["capacities"].reshape(28,)
B = scipy.io.loadmat('traffic.mat')["traffic"]
l = scipy.io.loadmat('traveltime.mat')["traveltime"].reshape(28,)

In [5]:

G = nx.DiGraph()
n_nodes = B.shape[0]
n_edges = B.shape[1]

G.add_nodes_from(range(n_nodes))

for edge_idx in range(n_edges):
    tail = np.where(B[:, edge_idx] == 1)[0][0]
    head = np.where(B[:, edge_idx] == -1)[0][0]
    
    G.add_edge(tail, head, weight=l[edge_idx], edge_index=edge_idx)

In [6]:

source = 0  
target = 16  

shortest_path = nx.shortest_path(G, source=source, target=target, weight='weight')
shortest_path_length = nx.shortest_path_length(G, source=source, target=target, weight='weight')

print(f"Shortest path from node {source+1} to node {target+1}:")
print(f"Path: {[n+1 for n in shortest_path]}")
print(f"Total travel time: {shortest_path_length:.4f}")

Shortest path from node 1 to node 17:
Path: [1, np.int64(2), np.int64(3), np.int64(9), np.int64(13), 17]
Total travel time: 0.5598


In [7]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.io 
import scipy
import cvxpy as cp
%matplotlib inline

file = scipy.io.loadmat('capacities.mat')
capacities = file.get('capacities')
capacities = capacities.reshape(28,)

file = scipy.io.loadmat('traveltime.mat')
traveltime = file.get('traveltime')
traveltime = traveltime.reshape(28,)

file = scipy.io.loadmat('flow.mat')["flow"]
flow = file.reshape(28,)

file = scipy.io.loadmat('traffic.mat')
traffic = file.get('traffic')

G = nx.DiGraph()
    
for c in range(28):
    capac = capacities[c]
    travtime = traveltime[c]
    for r in range(17):
        if traffic[r][c] == 1:
            i = r
        if traffic[r][c] == -1:
            j = r
    G.add_edges_from([(i+1, j+1)], capacity=capac, traveltime=travtime)


    


In [8]:

nu = B @ f_data


for i, val in enumerate(nu):
    if abs(val) > 1e-10:
        print(f"  Node {i+1} (index {i}): {val:.6f}")

nu_1 = nu[0]
print(f"\nν₁ = {nu_1:.6f}")
print(f"ν₁₇ = {-nu_1:.6f}")

  Node 1 (index 0): 16806.000000
  Node 2 (index 1): 8570.000000
  Node 3 (index 2): 19448.000000
  Node 4 (index 3): 4957.000000
  Node 5 (index 4): -746.000000
  Node 6 (index 5): 4768.000000
  Node 7 (index 6): 413.000000
  Node 8 (index 7): -2.000000
  Node 9 (index 8): -5671.000000
  Node 10 (index 9): 1169.000000
  Node 11 (index 10): -5.000000
  Node 12 (index 11): -7131.000000
  Node 13 (index 12): -380.000000
  Node 14 (index 13): -7412.000000
  Node 15 (index 14): -7810.000000
  Node 16 (index 15): -3430.000000
  Node 17 (index 16): -23544.000000

ν₁ = 16806.000000
ν₁₇ = -16806.000000


In [9]:

nu_opt = np.zeros(n_nodes)
nu_opt[0] = nu_1  
nu_opt[16] = -nu_1  

f = cp.Variable(n_edges)


objective = cp.Minimize(
    cp.sum([l[e] * C[e] * cp.inv_pos(1 - f[e]/C[e]) for e in range(n_edges)])
    - cp.sum(l * C)
)

constraints = [
    B @ f == nu_opt,  # Flow conservation
    f >= 0,           # Non-negativity
]

prob = cp.Problem(objective, constraints)
social_opt_cost = prob.solve()

f_star = f.value
#print(f"f_star: {f_star}")

print(f"Optimal cost: {social_opt_cost:.6f}")
print(f"\nOptimal flow distribution:")
for e in range(n_edges):
    if f_star[e] > 1e-6:
        print(f"  Link {e+1}: f* = {f_star[e]:.4f}")

Optimal cost: 26142.669632

Optimal flow distribution:
  Link 1: f* = 6569.0706
  Link 2: f* = 5809.9536
  Link 3: f* = 3046.9895
  Link 4: f* = 3046.9882
  Link 5: f* = 10236.9294
  Link 6: f* = 4666.5455
  Link 7: f* = 3061.2075
  Link 8: f* = 2596.0254
  Link 9: f* = 3104.5482
  Link 10: f* = 759.1170
  Link 11: f* = 0.0055
  Link 12: f* = 2762.9586
  Link 13: f* = 0.0012
  Link 14: f* = 3046.9882
  Link 15: f* = 5570.3839
  Link 16: f* = 2893.8793
  Link 17: f* = 5040.9596
  Link 18: f* = 2364.4550
  Link 19: f* = 465.1876
  Link 20: f* = 2254.4370
  Link 21: f* = 3359.0656
  Link 22: f* = 5613.5026
  Link 23: f* = 2371.9221
  Link 24: f* = 0.0013
  Link 25: f* = 6346.1287
  Link 26: f* = 5418.9104
  Link 27: f* = 5040.9609
  Link 28: f* = 5040.9609


In [10]:

total_travel_time_opt = sum(f_star[e] * l[e] / (1 - f_star[e]/C[e]) for e in range(n_edges))
print(f"Total travel time at social optimum: {total_travel_time_opt:.6f}")

Total travel time at social optimum: 26142.669632


In [11]:

f_ward = cp.Variable(n_edges)

objective_ward = cp.Minimize(
    cp.sum([-l[e] * C[e] * cp.log(1 - f_ward[e]/C[e]) for e in range(n_edges)])
)

constraints_ward = [
    B @ f_ward == nu_opt,
    f_ward >= 0,
]

prob_ward = cp.Problem(objective_ward, constraints_ward)
ward_cost = prob_ward.solve()

f_0 = f_ward.value
print(f"f_0: {f_0}")
print(f"Objective value: {ward_cost:.6f}")
print(f"\nWardrop flow distribution:")
for e in range(n_edges):
    if f_0[e] > 1e-6:
        print(f"  Link {e+1}: f⁽⁰⁾ = {f_0[e]:.4f}")

f_0: [6.55753759e+03 6.30856340e+03 2.20066558e+03 2.20066555e+03
 1.02484624e+04 4.70667763e+03 2.85995820e+03 2.23226376e+03
 3.34995738e+03 2.48974187e+02 1.16858737e+01 4.09621195e+03
 2.54781857e-05 2.20066555e+03 5.54178477e+03 2.34334640e+03
 5.29413199e+03 2.09569362e+03 6.39380313e+02 2.97851835e+03
 2.98272667e+03 5.96124502e+03 2.52240564e+03 3.83801231e-05
 6.78879676e+03 4.72307120e+03 5.29413203e+03 5.29413203e+03]
Objective value: 15731.952639

Wardrop flow distribution:
  Link 1: f⁽⁰⁾ = 6557.5376
  Link 2: f⁽⁰⁾ = 6308.5634
  Link 3: f⁽⁰⁾ = 2200.6656
  Link 4: f⁽⁰⁾ = 2200.6656
  Link 5: f⁽⁰⁾ = 10248.4624
  Link 6: f⁽⁰⁾ = 4706.6776
  Link 7: f⁽⁰⁾ = 2859.9582
  Link 8: f⁽⁰⁾ = 2232.2638
  Link 9: f⁽⁰⁾ = 3349.9574
  Link 10: f⁽⁰⁾ = 248.9742
  Link 11: f⁽⁰⁾ = 11.6859
  Link 12: f⁽⁰⁾ = 4096.2119
  Link 13: f⁽⁰⁾ = 0.0000
  Link 14: f⁽⁰⁾ = 2200.6656
  Link 15: f⁽⁰⁾ = 5541.7848
  Link 16: f⁽⁰⁾ = 2343.3464
  Link 17: f⁽⁰⁾ = 5294.1320
  Link 18: f⁽⁰⁾ = 2095.6936
  Link 19: f⁽⁰⁾ = 6

In [12]:

total_travel_time_ward = sum(f_0[e] * l[e] / (1 - f_0[e]/C[e]) for e in range(n_edges))
print(f"Total travel time at Wardrop equilibrium: {total_travel_time_ward:.6f}")

PoA = total_travel_time_ward / total_travel_time_opt
print(f"\nPrice of Anarchy: {PoA:.6f}")

Total travel time at Wardrop equilibrium: 26495.296798

Price of Anarchy: 1.013489


In [13]:

omega = np.zeros(n_edges)
for e in range(n_edges):
    if f_star[e] > 1e-10:
        tau_prime = l[e] / C[e] * (1 - f_star[e]/C[e])**(-2)
        omega[e] = f_star[e] * tau_prime

print("Marginal tolls:")
for e in range(n_edges):
    if omega[e] > 1e-6:
        print(f"  Link {e+1}: ω = {omega[e]:.6f}")

Marginal tolls:
  Link 1: ω = 1.973134
  Link 2: ω = 0.193178
  Link 3: ω = 0.049433
  Link 4: ω = 0.100107
  Link 5: ω = 1.512901
  Link 6: ω = 0.483692
  Link 7: ω = 0.112308
  Link 8: ω = 0.059651
  Link 9: ω = 0.271745
  Link 10: ω = 0.008285
  Link 12: ω = 0.067435
  Link 14: ω = 0.120076
  Link 15: ω = 0.498229
  Link 16: ω = 0.084657
  Link 17: ω = 0.073394
  Link 18: ω = 0.019181
  Link 19: ω = 0.001514
  Link 20: ω = 0.013113
  Link 21: ω = 0.067736
  Link 22: ω = 0.257076
  Link 23: ω = 0.066937
  Link 25: ω = 0.391859
  Link 26: ω = 0.269505
  Link 27: ω = 0.147695
  Link 28: ω = 0.597997


In [14]:

f_toll = cp.Variable(n_edges)

objective_toll = cp.Minimize(
    cp.sum([-l[e] * C[e] * cp.log(1 - f_toll[e]/C[e]) + omega[e] * f_toll[e] 
            for e in range(n_edges)])
)

constraints_toll = [
    B @ f_toll == nu_opt,
    f_toll >= 0,
]

prob_toll = cp.Problem(objective_toll, constraints_toll)
toll_cost = prob_toll.solve()

f_omega = f_toll.value
print(f"f_omega: {f_omega}")
print(f"\nComparison of flows:")
print(f"{'Link':<6} {'f*':<12} {'f⁽ω⁾':<12} {'Difference':<12}")
print("-" * 50)
for e in range(n_edges):
    if max(f_star[e], f_omega[e]) > 1e-6:
        diff = f_omega[e] - f_star[e]
        print(f"{e+1:<6} {f_star[e]:<12.6f} {f_omega[e]:<12.6f} {diff:<12.6f}")


f_omega: [6.57005701e+03 5.81018892e+03 3.04700374e+03 3.04700366e+03
 1.02359429e+04 4.66615312e+03 3.06150900e+03 2.59578825e+03
 3.10440391e+03 7.59868085e+02 2.54704443e-04 2.76318492e+03
 8.67350102e-05 3.04700366e+03 5.56978981e+03 2.89337396e+03
 5.04092806e+03 2.36451221e+03 4.65721002e+02 2.25456935e+03
 3.35909484e+03 5.61366419e+03 2.37210359e+03 1.22765100e-04
 6.34596452e+03 5.41910724e+03 5.04092818e+03 5.04092818e+03]

Comparison of flows:
Link   f*           f⁽ω⁾         Difference  
--------------------------------------------------
1      6569.070616  6570.057006  0.986390    
2      5809.953596  5810.188921  0.235324    
3      3046.989450  3047.003744  0.014293    
4      3046.988237  3047.003657  0.015419    
5      10236.929384 10235.942936 -0.986448   
6      4666.545507  4666.153121  -0.392386   
7      3061.207520  3061.509000  0.301480    
8      2596.025442  2595.788253  -0.237188   
9      3104.548240  3104.403910  -0.144330   
10     759.117020   759.868085

In [17]:
import numpy as np
import cvxpy as cp

f_star = cp.Variable(n_edges)

cost_terms = []
for e in range(n_edges):
    cost_terms.append(l[e] * cp.quad_over_lin(f_star[e], C[e] - f_star[e]))

objective_star = cp.Minimize(cp.sum(cost_terms))

constraints_star = [
    B @ f_star == nu_opt,
    f_star >= 0,
]

prob_star = cp.Problem(objective_star, constraints_star)
cost_star = prob_star.solve(solver=cp.SCS, verbose=False)
f_star_value = f_star.value
print(f"f_star: {f_star_value}")

print(f"Optimal cost: {cost_star:.6f}")


omega_star = np.zeros(n_edges)

for e in range(n_edges):
    f_e = f_star_value[e]
    c_e = C[e]
    l_e = l[e]
    
    tau_prime_e = l_e * c_e / ((c_e - f_e) ** 2)
    
    omega_star[e] = f_e * tau_prime_e - l_e


f_wardrop = cp.Variable(n_edges)

wardrop_cost = cp.sum(cp.multiply(omega_star, f_wardrop) - 
                       cp.multiply(C * l, cp.log(1 - cp.multiply(f_wardrop, 1/C))))

objective_wardrop = cp.Minimize(wardrop_cost)

constraints_wardrop = [
    B @ f_wardrop == nu_opt,
    f_wardrop >= 0,
 
]

prob_wardrop = cp.Problem(objective_wardrop, constraints_wardrop)
wardrop_result = prob_wardrop.solve(verbose=False)
f_wardrop_value = f_wardrop.value
print(f"f_wardrop: {f_wardrop_value}")

differences = np.abs(f_wardrop_value - f_star_value)
max_abs_diff = np.max(differences)

print(f"Max absolute difference: {max_abs_diff: .2e}")

f_star: [6.58443156e+03 5.57794805e+03 3.36742378e+03 3.36742375e+03
 1.02215685e+04 4.66947019e+03 3.15641576e+03 2.71179603e+03
 2.98794987e+03 1.00648351e+03 5.69996118e-05 2.21052421e+03
 2.88510642e-05 3.36742375e+03 5.55209828e+03 3.08472672e+03
 4.98690950e+03 2.51953794e+03 4.44619787e+02 1.93437040e+03
 3.52934643e+03 5.46371683e+03 2.20234566e+03 6.65149526e-05
 6.24932103e+03 5.56976941e+03 4.98690956e+03 4.98690956e+03]
Optimal cost: 15350.353947
f_wardrop: [6.58446762e+03 5.57791600e+03 3.36743201e+03 3.36743167e+03
 1.02215323e+04 4.66944419e+03 3.15644918e+03 2.71177372e+03
 2.98771243e+03 1.00655161e+03 2.97945465e-04 2.21048370e+03
 3.36381452e-04 3.36743167e+03 5.55208815e+03 3.08468064e+03
 4.98695414e+03 2.51954663e+03 4.44675750e+02 1.93454533e+03
 3.52935621e+03 5.46390154e+03 2.20239174e+03 1.83810370e-04
 6.24922223e+03 5.56982341e+03 4.98695432e+03 4.98695432e+03]
Max absolute difference:  2.37e-01
